In [3]:
import rouge

from tqdm import tqdm
from utils import read_lines

In [4]:
avg_evaluator = rouge.Rouge(metrics=['rouge-n', 'rouge-l', 'rouge-w'],
                            max_n=4,
                            limit_length=True,
                            length_limit=200,
                            length_limit_type='words',
                            apply_avg=True,
                            apply_best=False,
                            alpha=0.5, # Default F1_score
                            weight_factor=1.2,
                            stemming=True)

#### Read File

In [11]:
BEAM_SIZE = 6

In [17]:
xsum_train_preds_bm6_path = 'preds/xsum_train_bm6_all.hypo'
xsum_train_target_path = '/home/ml/cadencao/XSum/fairseq_files/train.target'

In [18]:
xsum_train_preds_bm6 = read_lines(xsum_train_preds_bm6_path)
xsum_train_target = read_lines(xsum_train_target_path)

In [22]:
xsum_train_preds_bm6 = [xsum_train_preds_bm6[i::BEAM_SIZE] for i in range(BEAM_SIZE)]

In [23]:
assert len(xsum_train_preds_bm6[0]) == len(xsum_train_target), "{}/{}".format(len(xsum_train_preds_bm6[0]), len(xsum_train_target))

#### Select Best Beam

In [24]:
index, best_predictions = 0, []
for t in tqdm(xsum_train_target):
    best_rouge_1, best_pred = -1.0, None
    for s in range(BEAM_SIZE):
        p = xsum_train_preds_bm6[s][index]
        score = avg_evaluator.get_scores([p], [t])
        if score['rouge-1']['f'] > best_rouge_1:
            best_rouge_1 = score['rouge-1']['f']
            best_pred = p
    best_predictions.append(best_pred)
    index += 1

100%|██████████| 203575/203575 [1:24:44<00:00, 40.04it/s]


In [25]:
assert len(best_predictions) == len(xsum_train_target)

In [26]:
with open('preds/xsum_train_best_beam.hypo', 'w') as f:
    for p in best_predictions:
        f.write(p + '\n')